In [1]:
import json

intents = {
    "intents": [
        {
            "tag": "greeting",
            "patterns": ["Hi", "Hello", "Hey", "Good morning", "Good evening"],
            "responses": ["Hello!", "Hi there!", "Hey! How can I help?"]
        },
        {
            "tag": "goodbye",
            "patterns": ["Bye", "See you later", "Goodbye"],
            "responses": ["Goodbye!", "Take care!", "See you later!"]
        },
        {
            "tag": "thanks",
            "patterns": ["Thanks", "Thank you", "That's helpful"],
            "responses": ["You're welcome!", "Glad to help!", "Anytime!"]
        },
        {
            "tag": "noanswer",
            "patterns": [],
            "responses": ["I'm not sure I understand. Can you rephrase?"]
        },
        {
            "tag": "play_song",
            "patterns": ["Play a song", "Can you play Believer?", "Play music"],
            "responses": ["Playing your song", "Here's something nice for you"]
        }
    ]
}

# Saving to file
with open("intents.json", "w") as f:
    json.dump(intents, f, indent=4)


In [2]:
import nltk
import numpy as np
import torch
import json
from nltk.stem.porter import PorterStemmer

In [3]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
stemmer = PorterStemmer() #creating a stemmer

In [5]:
def tokenize(sentence):
    return nltk.word_tokenize(sentence) #tokenizing the sentence

In [6]:
# Stemming the words
def stem(word):
    return stemmer.stem(word.lower())

In [8]:
def bag_of_words(tokenized_sentence, all_words):
    tokenized_sentence = [stem(w) for w in tokenized_sentence]
    bag = np.zeros(len(all_words), dtype=np.float32)
    for idx, w in enumerate(all_words):
        if w in tokenized_sentence:
            bag[idx] = 1.0
    return bag #bagging of words / vectorizing them for a better input feed

In [9]:
with open("intents.json", "r") as f:
    intents = json.load(f) #loading the file

In [10]:
all_words = []
tags = []
xy = []

In [11]:
# Tokenize and build word/tag lists
for intent in intents['intents']:
    tag = intent['tag']
    tags.append(tag)
    for pattern in intent['patterns']:
        w = tokenize(pattern)
        all_words.extend(w)
        xy.append((w, tag))

In [12]:
# Removing the punctuation & stem
ignore_words = ['?', '!', '.', ',']
all_words = [stem(w) for w in all_words if w not in ignore_words]
all_words = sorted(set(all_words))
tags = sorted(set(tags))

In [13]:
# Preparing training data
X_train = []
y_train = []

In [14]:
for (pattern_sentence, tag) in xy:
    bag = bag_of_words(pattern_sentence, all_words)
    X_train.append(bag)
    y_train.append(tags.index(tag))

X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long) #training the bag of words 

C:\Users\HP\AppData\Local\Temp\ipykernel_5704\3219632789.py:6: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\utils\tensor_new.cpp:257.)
  X_train = torch.tensor(X_train, dtype=torch.float32)


In [15]:
import torch.nn as nn
#wrapping the neural network to torch
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        return out

In [16]:
from torch.utils.data import Dataset, DataLoader

class ChatDataset(Dataset):
    def __init__(self):
        self.n_samples = len(X_train)
        self.x_data = X_train
        self.y_data = y_train

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.n_samples

In [17]:
#  Assigning corresponding Hyperparameters
batch_size = 8
hidden_size = 8
input_size = len(all_words)
output_size = len(tags)
learning_rate = 0.001
num_epochs = 400

In [18]:
dataset = ChatDataset()
train_loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True)

In [19]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = NeuralNet(input_size, hidden_size, output_size).to(device)

In [21]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss() #using cross entropy as loss caluculator
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [22]:
for epoch in range(num_epochs):
    for (words, labels) in train_loader:
        words = words.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(words)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()#back propagation
        optimizer.step()

    # Printing every 10 epochs
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

print(f"Final Loss: {loss.item():.4f}")


Epoch [10/400], Loss: 1.5630
Epoch [20/400], Loss: 1.4812
Epoch [30/400], Loss: 1.4634
Epoch [40/400], Loss: 1.4358
Epoch [50/400], Loss: 1.3289
Epoch [60/400], Loss: 1.2693
Epoch [70/400], Loss: 1.1855
Epoch [80/400], Loss: 1.3592
Epoch [90/400], Loss: 1.1623
Epoch [100/400], Loss: 0.8948
Epoch [110/400], Loss: 0.8630
Epoch [120/400], Loss: 0.9498
Epoch [130/400], Loss: 0.8022
Epoch [140/400], Loss: 0.9316
Epoch [150/400], Loss: 0.7270
Epoch [160/400], Loss: 0.5691
Epoch [170/400], Loss: 0.5887
Epoch [180/400], Loss: 0.3815
Epoch [190/400], Loss: 0.4619
Epoch [200/400], Loss: 0.4025
Epoch [210/400], Loss: 0.4339
Epoch [220/400], Loss: 0.5439
Epoch [230/400], Loss: 0.3881
Epoch [240/400], Loss: 0.3425
Epoch [250/400], Loss: 0.3604
Epoch [260/400], Loss: 0.2648
Epoch [270/400], Loss: 0.3112
Epoch [280/400], Loss: 0.2292
Epoch [290/400], Loss: 0.2174
Epoch [300/400], Loss: 0.1930
Epoch [310/400], Loss: 0.1914
Epoch [320/400], Loss: 0.1830
Epoch [330/400], Loss: 0.0909
Epoch [340/400], Lo

In [23]:
data = {"model_state": model.state_dict(),"input_size": input_size,
    "hidden_size": hidden_size,"output_size": output_size,
    "all_words": all_words,"tags": tags} #dictionary of the neural network sizes

FILE = "trained_model.pth"
torch.save(data, FILE) #saving the model built with torch

print(f"Model trained and saved to {FILE}")


Model trained and saved to trained_model.pth


In [24]:
import torch.nn as nn

class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        return out
FILE = "trained_model.pth"
data = torch.load(FILE) #unpacking the built blue print of the model to apply the content of the bot

input_size = data["input_size"]
hidden_size = data["hidden_size"]
output_size = data["output_size"]
all_words = data["all_words"]
tags = data["tags"]
model_state = data["model_state"]

model = NeuralNet(input_size, hidden_size, output_size)
model.load_state_dict(model_state)
model.eval()


NeuralNet(
  (l1): Linear(in_features=21, out_features=8, bias=True)
  (relu): ReLU()
  (l2): Linear(in_features=8, out_features=5, bias=True)
)

In [27]:
import json
import random

# Load intents
with open("intents.json", "r") as f:
    intents = json.load(f)

def get_response(msg):
    sentence = tokenize(msg) #tokenizing the input
    X = bag_of_words(sentence, all_words) # converting the word to a vector for better prediction 
    X = X.reshape(1, X.shape[0])
    X = torch.from_numpy(X).float()

    output = model(X) #feeding it to the model (NN)
    _, predicted = torch.max(output, dim=1)
    tag = tags[predicted.item()]

    probs = torch.softmax(output, dim=1)
    prob = probs[0][predicted.item()]
    if prob.item() > 0.3: #if the propbability is > .3 then go to that particular intent and return random response from the respones of that intent
        for intent in intents["intents"]:
            if tag == intent["tag"]:
                return random.choice(intent["responses"])
    else:
        return "I'm not sure I understand. Can you rephrase?"

In [28]:
!pip install gradio

     ---------------------------------------- 0.0/43.0 kB ? eta -:--:--
     ---------------------------------------- 43.0/43.0 kB 2.2 MB/s eta 0:00:00
     ---------------------------------------- 0.0/68.0 kB ? eta -:--:--
     ---------------------------------------- 68.0/68.0 kB 3.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/54.3 MB ? eta -:--:--
   ---------------------------------------- 0.4/54.3 MB 12.2 MB/s eta 0:00:05
    --------------------------------------- 0.8/54.3 MB 9.7 MB/s eta 0:00:06
    --------------------------------------- 0.9/54.3 MB 7.0 MB/s eta 0:00:08
    --------------------------------------- 1.0/54.3 MB 6.5 MB/s eta 0:00:09
   - -------------------------------------- 1.4/54.3 MB 6.5 MB/s eta 0:00:09
   - -------------------------------------- 1.8/54.3 MB 7.0 MB/s eta 0:00:08
   - -------------------------------------- 1.8/54.3 MB 6.4 MB/s eta 0:00:09
   - -------------------------------------- 2.4/54.3 MB 7.1 MB/s eta 0:00:08
   - ----

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
anaconda-cloud-auth 0.1.4 requires pydantic<2.0, but you have pydantic 2.11.7 which is incompatible.


Using gradio to get the look and feel of the bot.

In [30]:
import gradio as gr

def chatbot_interface(message, history):
    try:
        if message.lower() == "quit":
            gr.close_all() 
            return "Chat ended. Goodbye!"
        
        response = get_response(message)
        return response

    except Exception as e:
        return f" Error: {str(e)}"


quit--> closes or end the chat
response takes the message and feeds get_response() that inturn feeds the model and returns response
error--> if nothing is from the .json file -- intent/tags

In [31]:
import gradio as gr

chatbot = gr.ChatInterface(fn=chatbot_interface, title="ChatBot", chatbot=[])
chatbot.launch()


C:\Users\HP\anaconda3\Lib\site-packages\gradio\chat_interface.py:339: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.
